import 函式庫

In [25]:
import glob
import json
import numpy as np
from tensorflow import keras
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dropout
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import OneHotEncoder
from keras.models import load_model

data processing

In [14]:
ALL_CHARACTERS = ['DEFECT', 'IRONCLAD', 'THE_SILENT', 'WATCHER']

In [15]:
def encode_single(value, category):
  np_array = np.array([[value]])
  encoder = OneHotEncoder(categories=[category], sparse=False)
  onehot_encoded = encoder.fit_transform(np_array)
  collapsed = np.sum(onehot_encoded, axis=0)
  # inverse = encoder.inverse_transform(collapsed[np.newaxis, ...])
  # print(np.array_equal(np_array, inverse))
  return collapsed

In [16]:
def encode_character(character):
  """
  Encode the chosen character into a one-hot vector of length ALL_CHARACTERS
  """
  return encode_single(character, ALL_CHARACTERS)

In [17]:
def encode_sample_with_loop(sample):
  """
  Encode a single sample into a 1D vector
  [{"cards": 10, "relics": 1, "ascension": 20, "character": "IRONCLAD", "floor": 0, "potions": 0, "path": "M", "max_hp": 82, "current_hp": 82, "gold": 99, "value": 2884.076040777472, "upgrade_cards": 0, "curse_cards": 1}
  """
  character=encode_character(sample['character'])
 # print(character)
  num_data = np.array([sample['cards'],sample['relics'],sample['ascension'],sample['floor'],sample['potions'],ord(sample['path']),sample['max_hp'],sample['current_hp'],sample['gold'],sample['upgrade_cards'],sample['curse_cards']])
  x=np.concatenate((character, num_data))
  #print(len(x))
  return  x

In [18]:
def preprocess_with_loop(data):
  preprocess_list=[]
  y=[]
  for i,sample in enumerate(data):
    if sample['path']!= None:
      preprocess_list.append(encode_sample_with_loop(sample))
      y.append(sample['value'])
  X=np.vstack(preprocess_list)
  #print(len(X))
  Y=np.array(y,dtype='float64')
  #print(len(Y))
  return X,Y

In [19]:
files=glob.glob('test_out/*',recursive=True)
count=0
data=[]
for single_file in files:
  with open(single_file,'r') as file:
    tmp=json.loads(file.read())
    #print(tmp)
    data.extend(tmp)
    X,Y=preprocess_with_loop(data)
print(len(X))
print(len(Y))
#print(X)
#print(Y)

3637
3637


In [20]:
def scale_X(X_data):
  """
  Used with one hot encoded model
  """
  X_copy = np.copy(X_data)
  max_abs_scaler = MaxAbsScaler()
  X_maxabs = max_abs_scaler.fit_transform(X_copy)
#  with open('input_scales.json', 'w') as out_file:
 #   json.dump(max_abs_scaler.scale_.tolist(), out_file)
  return X_maxabs

In [21]:
def scale_Y(Y_data):
  Y_copy = np.copy(Y_data)

  # Scale Y
  Y_copy /= 10

  # To allow healing (negative damage), uncomment `Y[Y < -1] = -1` and comment out `Y[Y < 0] = 0`
  #Y_copy[Y_copy < -1] = -1 # Healing (negative damage)
  # Y_copy[Y_copy < 0] = 0 # No healing

  # Cap damage taken at 100
  Y_copy[Y_copy > 100] = 100
  return Y_copy

artifitial nerural network using tensorflow  極有可能會修改

In [22]:
model=Sequential()
model.add(Dense(1024,input_shape=(15,),activation='relu')) 
model.add(Dropout(.8))
model.add(Dense(512,activation='relu'))
model.add(Dropout(.4))
model.add(Dense(256,activation='relu'))
model.add(Dropout(.2))
model.add(Dense(128,activation='relu'))
model.add(Dropout(.1))
model.add(Dense(64,activation='relu'))
model.add(Dense(1))
model.summary()
model.compile(
    optimizer=keras.optimizers.RMSprop(learning_rate=.001),
    loss='mean_absolute_error',
    metrics=['mean_absolute_error', 'mean_squared_error'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 1024)              16384     
_________________________________________________________________
dropout_4 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_6 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 128)              

train 

In [27]:
X_scale=scale_X(X)
Y_scale=scale_Y(Y)
X_train, X_test, Y_train, Y_test = train_test_split(X_scale, Y_scale, test_size=0.2, shuffle=True)
model=load_model('AI_FP_ROUTE.h5')
history = model.fit(X_train, Y_train, batch_size=32, epochs=100, validation_split=0.2)
a=model.evaluate(X_train,Y_train,verbose=2)
b=model.evaluate(X_test,Y_test,verbose=2)
print(b[0])
model.save('AI_FP_ROUTE.h5')


Epoch 1/100
73/73 [==============================] - 2s 12ms/step - loss: 5.2218 - mean_absolute_error: 5.2218 - mean_squared_error: 425.4871 - val_loss: 14.9409 - val_mean_absolute_error: 14.9409 - val_mean_squared_error: 461.9084
Epoch 2/100
73/73 [==============================] - 0s 7ms/step - loss: 5.2429 - mean_absolute_error: 5.2429 - mean_squared_error: 424.9565 - val_loss: 15.7273 - val_mean_absolute_error: 15.7273 - val_mean_squared_error: 474.0177
Epoch 3/100
73/73 [==============================] - 0s 6ms/step - loss: 5.2295 - mean_absolute_error: 5.2295 - mean_squared_error: 424.2676 - val_loss: 16.1220 - val_mean_absolute_error: 16.1220 - val_mean_squared_error: 480.6707
Epoch 4/100
73/73 [==============================] - 0s 7ms/step - loss: 5.2273 - mean_absolute_error: 5.2273 - mean_squared_error: 424.8380 - val_loss: 15.8996 - val_mean_absolute_error: 15.8996 - val_mean_squared_error: 476.8718
Epoch 5/100
73/73 [==============================] - 0s 6ms/step - loss: 5.